# Trajectory estimation

In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import os

%matplotlib notebook
%reload_ext autoreload
%autoreload 2

np.set_printoptions(precision=2)

## Read and plot dataset

In [ ]:
import pickle

dimension = 2
chosen_distance = 'distance_tango'#'distance_median_0'

out_dir = 'experiments/robot_test/'

name = 'circle2_double'
df = pd.read_pickle(out_dir + name + '_0_calibrated.pkl')
df.reset_index(inplace=True, drop=True)
df.timestamp -= df.timestamp.values[0]
df.head()

In [ ]:
tango_df = df[df.system_id=='Tango']
plt.figure()
sns.scatterplot(x='px', y='py', hue='timestamp', data=tango_df)
plt.title('2D of Tango data')
plt.xlabel('x [m]'); plt.ylabel('y [m]')
plt.figure()
plt.plot(tango_df.timestamp, tango_df.pz)
plt.title('z of Tango data')
plt.xlabel('time [s]'); plt.ylabel('z [m]')

In [ ]:
df_rtt0 = df[df.anchor_name == 'RTT 0']
plt.scatter(df_rtt0.timestamp, df_rtt0.loc[:, chosen_distance])

## Construct D, basis and anchors

In [ ]:
from evaluate_dataset import read_anchors_df
anchorsfile = 'experiments/anchors.csv'
anchors_df = read_anchors_df(anchorsfile)

# if we do not have all we could do something elegant with pandas. 
print(anchor_names)
anchors = anchors_df.loc[anchors_df.anchor_name.isin(anchor_names), ['px', 'py', 'pz']].values.T
print(anchors.shape)

In [ ]:
from trajectory import Trajectory
from global_variables import TAU

#from trajectory_creator import get_trajectory
#traj = get_trajectory(name + '.csv')
traj = Trajectory(n_complexity=3, dim=dimension, model='full_bandlimited')
traj.set_coeffs(seed=3)
robot_height = 0.25

In [ ]:
n_measurements = -1
# N_times x N_anchors
anchor_names = sorted(df[df.system_id=='RTT'].anchor_name.unique().tolist())
n_anchors = len(anchor_names)

times = df.iloc[:n_measurements].timestamp.unique().astype(np.float32)
n_times = len(times)
D_topright = np.zeros((n_times, n_anchors))

fig, ax = plt.subplots()
ax = traj.plot()
#lengths = tango_df.length.values[indices]
#distances = np.cumsum(lengths)
speed = 1/6. # 10m/min
distances = speed * times * TAU
times_corr, _, _  = traj.get_times_from_distances(arbitrary_distances=distances, plot=False)
    
basis = traj.get_basis(times=times_corr)
print(basis.shape)
points = traj.get_sampling_points(basis=basis)
ax.scatter(*points[:2]) 

## real measurements

In [ ]:
for i, t in enumerate(times):
    this_slice = df[(df.system_id=='RTT') & (df.timestamp == t)]
    
    # this can be done more elegantly with pandas
    for anchor_name in this_slice.anchor_name:
        a_id = anchor_names.index(anchor_name)
        distance = this_slice.loc[this_slice.anchor_name==anchor_name, chosen_distance].values[0]
        
        if dimension == 3:
            D_topright[i, a_id] = distance ** 2
        else:
            anchor_height = anchors_df[anchors_df.anchor_name==anchor_name].pz
            distance_sq = distance ** 2 - (anchor_height - robot_height) ** 2
            D_topright[i, a_id] = distance_sq
        
#plt.figure()
#plt.matshow(D_topright)

## simulated measurements

In [ ]:
if dimension == 3:
    coeffs = np.array([[0, 2, 0], [0, 0, 2], [robot_height, 0, 0]])
else:
    coeffs = np.array([[0, 2, 0], [0, 0, 2]])
    print(coeffs.shape)
traj.set_coeffs(coeffs=coeffs)
basis = traj.get_basis(times=times_corr)
points = traj.get_sampling_points(basis=basis)
plt.scatter(*points[:2])
plt.axis('equal')

from measurements import get_D_topright, add_noise
D_topright = get_D_topright(anchors=anchors[:dimension], samples=points)
sigma = 10#0.1
D_topright = add_noise(D_topright, sigma, noise_to_square=False)

In [ ]:
from constraints import get_C_constraints

anchors = anchors[:dimension]

# sanity check if dimensions etc. are correct
T_A, T_B, b = get_C_constraints(D_topright, anchors, basis)

## Estimate trajectory

In [ ]:
from solvers import alternativePseudoInverse
coeffs = alternativePseudoInverse(D_topright, anchors, basis, 
                                  weighted=False)
print(coeffs)
#print(traj.coeffs)
trajectory_estimated = traj.copy()
trajectory_estimated.set_coeffs(coeffs=coeffs)
ax = trajectory_estimated.plot()
points = trajectory_estimated.get_sampling_points(basis=basis)
points_true = traj.get_sampling_points(basis=basis)
ax.scatter(*points[:2])
ax.scatter(*points_true[:2])
#ax.scatter(*tango_df.loc[:, ['px', 'py']].values.T)
plt.axis('equal')
if dimension > 2:
    plt.figure()
    plt.plot(times, points[2])
    plt.plot(times, points_true[2])

When setting sigma to 10 in simulated measurements, then the recovered trajectory looks very similar to the reconstruction using real noisy measurements. We should investigate why. Hypothesis: we take samples at the same time so maybe we oversample in one region compared to the other region.  